In [1]:
import os


In [2]:
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # For implicit ALS

In [3]:
import warnings

warnings.filterwarnings('ignore')

In [4]:


import pandas as pd
import numpy as np

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools import Columns
from rectools.dataset import Dataset

import matplotlib.pyplot as plt
import typing as tp


# Acquire dataset

In [5]:
interactions = pd.read_csv('../kion_train/interactions.csv')
users = pd.read_csv('../kion_train/users.csv')
items = pd.read_csv('../kion_train/items.csv')


# Data info

---
### interactions

In [6]:
pd.concat([interactions.head(), interactions.tail()])


,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0
5476250,319709,4436,2021-08-15,3921,45.0


In [7]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   user_id        int64  
 1   item_id        int64  
 2   last_watch_dt  object 
 3   total_dur      int64  
 4   watched_pct    float64
dtypes: float64(1), int64(3), object(1)
memory usage: 208.9+ MB


---
### items

In [8]:
pd.concat([items.head(), items.tail()])


,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."
15958,6443,series,Полярный круг,Arctic Circle,2018.0,"драмы, триллеры, криминал","Финляндия, Германия",NaN,16.0,NaN,Ханну Салонен,"Иина Куустонен, Максимилиан Брюкнер, Пихла Вии...","Во время погони за браконьерами по лесу, сотру...","убийство, вирус, расследование преступления, н..."
15959,2367,series,Надежда,NaN,2020.0,"драмы, боевики",Россия,0.0,18.0,NaN,Елена Хазанова,"Виктория Исакова, Александр Кузьмин, Алексей М...",Оригинальный киносериал от создателей «Бывших»...,"Надежда, 2020, Россия"
15960,10632,series,Сговор,Hassel,2017.0,"драмы, триллеры, криминал",Россия,0.0,18.0,NaN,"Эшреф Рейбрук, Амир Камдин, Эрик Эгер","Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Р...",Криминальная драма по мотивам романов о шведск...,"Сговор, 2017, Россия"
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родд...",Семнадцатилетний Дэмиен мечтает вырваться за п...,"Среди, камней, 2019, Россия"
15962,3206,series,Гоша,NaN,2019.0,комедии,Россия,0.0,16.0,NaN,Михаил Миронов,"Мкртыч Арзуманян, Виктория Рунцова","Добродушный Гоша не может выйти из дома, чтобы...","Гоша, 2019, Россия"


In [9]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15963 entries, 0 to 15962
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   item_id       15963 non-null  int64  
 1   content_type  15963 non-null  object 
 2   title         15963 non-null  object 
 3   title_orig    11218 non-null  object 
 4   release_year  15865 non-null  float64
 5   genres        15963 non-null  object 
 6   countries     15926 non-null  object 
 7   for_kids      566 non-null    float64
 8   age_rating    15961 non-null  float64
 9   studios       1065 non-null   object 
 10  directors     14454 non-null  object 
 11  actors        13344 non-null  object 
 12  description   15961 non-null  object 
 13  keywords      15540 non-null  object 
dtypes: float64(3), int64(1), object(10)
memory usage: 1.7+ MB


---
### users

In [10]:
pd.concat([users.head(), users.tail()])


,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
840192,339025,age_65_inf,income_0_20,Ж,0
840193,983617,age_18_24,income_20_40,Ж,1
840194,251008,NaN,NaN,NaN,0
840195,590706,NaN,NaN,Ж,0
840196,166555,age_65_inf,income_20_40,Ж,0


In [11]:
users.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840197 entries, 0 to 840196
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   840197 non-null  int64 
 1   age       826102 non-null  object
 2   income    825421 non-null  object
 3   sex       826366 non-null  object
 4   kids_flg  840197 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 32.1+ MB


# Fake users

In [14]:
max_id = users["user_id"].max()
print(max_id)

1097558


In [15]:
fake_user_ids = []

###  Любитель триллеров
Так как одна из фич для айтемов - это жанр, проставим пользователю фильмы с жанром `триллер`

In [16]:
new_user_id = max_id + 1
fake_user_ids.append(new_user_id)
max_id += 1

In [17]:

thriller_ids = items[(items["genres"]).str.contains("триллер")]["item_id"]
thriller_ids

2        10716
8         9853
17        6881
21        3445
38        1622
         ...  
15923    10288
15939    15699
15945    12880
15958     6443
15960    10632
Name: item_id, Length: 2418, dtype: int64

In [18]:
thriller_ids_sample = thriller_ids.sample(n=20)

In [19]:
last_watch_dt = ["2021-04-13"] * 19
last_watch_dt.append("2021-08-22")
total_dur = np.round(np.random.rand(20) * 10000 + 100).astype(dtype=np.int)
watched_pct = np.round(np.random.rand(20) * 100).astype(dtype=np.int)
data = {
    "user_id": [new_user_id] * 20,
    "item_id": list(thriller_ids_sample),
    "last_watch_dt": last_watch_dt,
    "total_dur": list(total_dur),
    "watched_pct": list(watched_pct)
}
new_interactions = pd.DataFrame(data)
interactions = interactions.append(new_interactions)
new_interactions

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,1097559,8226,2021-04-13,4824,64
1,1097559,11914,2021-04-13,6483,74
2,1097559,4303,2021-04-13,7820,75
3,1097559,828,2021-04-13,9121,72
4,1097559,9140,2021-04-13,4144,93
5,1097559,4546,2021-04-13,7231,30
6,1097559,9444,2021-04-13,6984,83
7,1097559,6309,2021-04-13,9665,55
8,1097559,16133,2021-04-13,9794,60
9,1097559,2842,2021-04-13,4893,97


In [20]:
pd.concat([interactions.head(), interactions.tail()])


,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
15,1097559,134,2021-04-13,4520,35.0
16,1097559,16425,2021-04-13,6164,47.0
17,1097559,14179,2021-04-13,9525,95.0
18,1097559,14056,2021-04-13,6590,87.0
19,1097559,15817,2021-08-22,1005,20.0


In [21]:
data = {
    "user_id": [new_user_id],
    "age": ["age_25_34"],
    "income": ["income_60_90"],
    "sex": "М",
    "kids_flg": [1]
}

new_user = pd.DataFrame(data)
users = users.append(new_user)
new_user

,user_id,age,income,sex,kids_flg
0,1097559,age_25_34,income_60_90,М,1


In [22]:
users[users["user_id"] == new_user_id]

,user_id,age,income,sex,kids_flg
0,1097559,age_25_34,income_60_90,М,1


###  Поклонник спорта
По аналогии с предыдущим, найдем фильмы с жанром `спорт`

In [23]:
new_user_id = max_id + 1
fake_user_ids.append(new_user_id)
max_id += 1

In [24]:
sport_ids = items[(items["genres"]).str.contains("спорт")]["item_id"]
sport_ids

4        16268
35       15702
221      15935
272        849
340       4639
         ...  
15856    14273
15868     6467
15920     9035
15938     3134
15961     4538
Name: item_id, Length: 308, dtype: int64

In [25]:
sport_ids_sample = sport_ids.sample(n=100)

In [26]:
last_watch_dt = ["2021-04-13"] * 99
last_watch_dt.append("2021-08-22")
total_dur = np.round(np.random.rand(100) * 10000 + 100).astype(dtype=np.int)
watched_pct = np.round(np.random.rand(100) * 100).astype(dtype=np.int)
data = {
    "user_id": [new_user_id] * 100,
    "item_id": list(sport_ids_sample),
    "last_watch_dt": last_watch_dt,
    "total_dur": list(total_dur),
    "watched_pct": list(watched_pct)
}

new_interactions = pd.DataFrame(data)
interactions = interactions.append(new_interactions)
new_interactions

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,1097560,12500,2021-04-13,5068,76
1,1097560,11781,2021-04-13,9638,12
2,1097560,14141,2021-04-13,3679,35
3,1097560,7088,2021-04-13,2426,65
4,1097560,700,2021-04-13,7570,91
...,...,...,...,...,...
95,1097560,9818,2021-04-13,4226,71
96,1097560,14263,2021-04-13,1064,12
97,1097560,13919,2021-04-13,1797,3
98,1097560,9873,2021-04-13,463,44


In [27]:
data = {
    "user_id": [new_user_id],
    "age": ["age_18_24"],
    "income": ["income_20_40"],
    "sex": ["М"],
    "kids_flg": [0]
}

new_user = pd.DataFrame(data)
users = users.append(new_user)
new_user

,user_id,age,income,sex,kids_flg
0,1097560,age_18_24,income_20_40,М,0


In [28]:
users[users["user_id"] == new_user_id]

,user_id,age,income,sex,kids_flg
0,1097560,age_18_24,income_20_40,М,0


###  Обожает сериалы
Теперь абстрагируемся от жанра и укажем, что пользователь смотрит только сериалы

In [29]:
new_user_id = max_id + 1
fake_user_ids.append(new_user_id)
max_id += 1

In [30]:
series_ids = items[(items["content_type"]).str.contains("series")]["item_id"]
series_ids

76        3965
77        4045
78        2662
79        5297
80        4084
         ...  
15958     6443
15959     2367
15960    10632
15961     4538
15962     3206
Name: item_id, Length: 3961, dtype: int64

In [31]:
series_ids_sample = series_ids.sample(n=100)

In [32]:
last_watch_dt = ["2021-04-13"] * 99
last_watch_dt.append("2021-08-22")
total_dur = np.round(np.random.rand(100) * 10000 + 100).astype(dtype=np.int)
watched_pct = np.round(np.random.rand(100) * 100).astype(dtype=np.int)
data = {
    "user_id": [new_user_id] * 100,
    "item_id": list(series_ids_sample),
    "last_watch_dt": last_watch_dt,
    "total_dur": list(total_dur),
    "watched_pct": list(watched_pct)
}

new_interactions = pd.DataFrame(data)
interactions = interactions.append(new_interactions)
new_interactions

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,1097561,3932,2021-04-13,1275,33
1,1097561,15263,2021-04-13,9285,44
2,1097561,2824,2021-04-13,8312,9
3,1097561,1514,2021-04-13,7855,57
4,1097561,12901,2021-04-13,828,9
...,...,...,...,...,...
95,1097561,16058,2021-04-13,1455,38
96,1097561,16322,2021-04-13,8986,27
97,1097561,16121,2021-04-13,6446,76
98,1097561,12181,2021-04-13,5362,54


In [33]:
data = {
    "user_id": [new_user_id],
    "age": ["age_45_54"],
    "income": ["income_40_60"],
    "sex": ["Ж"],
    "kids_flg": [1]
}

new_user = pd.DataFrame(data)
users = users.append(new_user)
new_user

,user_id,age,income,sex,kids_flg
0,1097561,age_45_54,income_40_60,Ж,1


In [34]:
users[users["user_id"] == new_user_id]

,user_id,age,income,sex,kids_flg
0,1097561,age_45_54,income_40_60,Ж,1


# Preprocessing

In [35]:
Columns.Datetime = 'last_watch_dt'

In [36]:
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)

In [39]:
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

In [40]:
max_date = interactions[Columns.Datetime].max()

In [41]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 30, 5, 1)

In [42]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985486, 6)
test: (490985, 6)


In [43]:
cold_users = set(test[Columns.User]) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

# Features

### Users

In [44]:
users.isnull().sum()
users.nunique()

user_id     840200
age              6
income           6
sex              2
kids_flg         2
dtype: int64

In [45]:
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()

In [46]:
users

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
5,1037719,age_45_54,income_60_90,М,0
...,...,...,...,...,...
840192,339025,age_65_inf,income_0_20,Ж,0
840194,251008,NaN,NaN,NaN,0
0,1097559,age_25_34,income_60_90,М,1
0,1097560,age_18_24,income_20_40,М,0


In [47]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
pd.concat([user_features.head(), user_features.tail()])

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex
840192,339025,income_0_20,income
840194,251008,NaN,income
0,1097559,income_60_90,income
0,1097560,income_20_40,income
0,1097561,income_40_60,income


### Items

In [48]:
items.isnull().sum()

item_id             0
content_type        0
title               0
title_orig       4745
release_year       98
genres              0
countries          37
for_kids        15397
age_rating          2
studios         14898
directors        1509
actors           2619
description         2
keywords          423
dtype: int64

In [49]:
items.nunique()

item_id         15963
content_type        2
title           15293
title_orig      10575
release_year      105
genres           2771
countries         687
for_kids            2
age_rating          6
studios            38
directors        7973
actors          12996
description     15619
keywords        15492
dtype: int64

In [50]:
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()

In [51]:
items

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15958,6443,series,Полярный круг,Arctic Circle,2018.0,"драмы, триллеры, криминал","Финляндия, Германия",NaN,16.0,NaN,Ханну Салонен,"Иина Куустонен, Максимилиан Брюкнер, Пихла Вии...","Во время погони за браконьерами по лесу, сотру...","убийство, вирус, расследование преступления, н..."
15959,2367,series,Надежда,NaN,2020.0,"драмы, боевики",Россия,0.0,18.0,NaN,Елена Хазанова,"Виктория Исакова, Александр Кузьмин, Алексей М...",Оригинальный киносериал от создателей «Бывших»...,"Надежда, 2020, Россия"
15960,10632,series,Сговор,Hassel,2017.0,"драмы, триллеры, криминал",Россия,0.0,18.0,NaN,"Эшреф Рейбрук, Амир Камдин, Эрик Эгер","Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Р...",Криминальная драма по мотивам романов о шведск...,"Сговор, 2017, Россия"
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родд...",Семнадцатилетний Дэмиен мечтает вырваться за п...,"Среди, камней, 2019, Россия"


#### Genre

In [52]:
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


#### Content

In [53]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"
content_feature.head()

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type
3,7868,film,content_type
4,16268,film,content_type


In [54]:
item_features = pd.concat((genre_feature, content_feature))

In [55]:
pd.concat([item_features.head(), item_features.tail()])

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre
15958,6443,series,content_type
15959,2367,series,content_type
15960,10632,series,content_type
15961,4538,series,content_type
15962,3206,series,content_type


# Metrics

In [56]:
metrics_name = {
    'Precision': Precision,
    'Recall': Recall,
    'MAP': MAP,
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in range(1, 11):
        metrics[f'{metric_name}@{k}'] = metric(k=k)

In [57]:
metrics

{'Precision@1': Precision(k=1),
 'Precision@2': Precision(k=2),
 'Precision@3': Precision(k=3),
 'Precision@4': Precision(k=4),
 'Precision@5': Precision(k=5),
 'Precision@6': Precision(k=6),
 'Precision@7': Precision(k=7),
 'Precision@8': Precision(k=8),
 'Precision@9': Precision(k=9),
 'Precision@10': Precision(k=10),
 'Recall@1': Recall(k=1),
 'Recall@2': Recall(k=2),
 'Recall@3': Recall(k=3),
 'Recall@4': Recall(k=4),
 'Recall@5': Recall(k=5),
 'Recall@6': Recall(k=6),
 'Recall@7': Recall(k=7),
 'Recall@8': Recall(k=8),
 'Recall@9': Recall(k=9),
 'Recall@10': Recall(k=10),
 'MAP@1': MAP(k=1, divide_by_k=False),
 'MAP@2': MAP(k=2, divide_by_k=False),
 'MAP@3': MAP(k=3, divide_by_k=False),
 'MAP@4': MAP(k=4, divide_by_k=False),
 'MAP@5': MAP(k=5, divide_by_k=False),
 'MAP@6': MAP(k=6, divide_by_k=False),
 'MAP@7': MAP(k=7, divide_by_k=False),
 'MAP@8': MAP(k=8, divide_by_k=False),
 'MAP@9': MAP(k=9, divide_by_k=False),
 'MAP@10': MAP(k=10, divide_by_k=False)}

# Models

In [58]:
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)
K_RECOS = 10
SEED = 123
NUM_THREADS = 16
N_FACTORS = (32, 64)
N_EPOCHS = 1
USER_ALPHA = 0
ITEM_ALPHA = 0
LEARNING_RATE = (0.05, 0.01)


In [59]:
models = {}

In [60]:
%%time
from service.rec_models.als_model import ALSModel, ALSFeatures

for is_fitting_features in (True, False):
    for n_factors in N_FACTORS:
        print(f"ALS_{n_factors}_{is_fitting_features}")
        models[f"ALS_{n_factors}_{is_fitting_features}"] = (
            ALSModel(
                train=train,
                n_factors=n_factors,
                is_fitting_features=is_fitting_features,
                features=ALSFeatures(
                  user_features_df=user_features,
                    cat_user_features=["sex", "age", "income"],
                    item_features_df=item_features,
                    cat_item_features=["genre", "content_type"]
                ),
                n_threads=NUM_THREADS,
                seed=SEED
            )
        )

ALS_32_True
ALS_64_True
ALS_32_False
ALS_64_False
CPU times: user 23.6 s, sys: 1min 15s, total: 1min 38s
Wall time: 1min 39s


In [61]:
%%time
from service.rec_models.factoring_machine_model import FactoringMachineModel, FMHyperParams, FMTuneParams, FMFeatures, \
    NeighParams

for n_factors in N_FACTORS:
    for lr in LEARNING_RATE:
        print(f"LightFM_{n_factors}_{lr}")
        models[f"LightFM_{n_factors}_{lr}"] = (
            FactoringMachineModel(
                train=train,
                hyper_params = FMHyperParams(
                    n_factors=n_factors,
                    loss="warp",
                    lr=lr,
                    ua=USER_ALPHA,
                    ia=ITEM_ALPHA,
                ),
                tune_params = FMTuneParams(
                    n_epoch=N_EPOCHS,
                    n_threads=NUM_THREADS,
                    seed=SEED
                ),
                neigh_params=NeighParams(),
                features=FMFeatures(
                    user_features_df=user_features,
                    cat_user_features=["sex", "age", "income"],
                    item_features_df=item_features,
                    cat_item_features=["genre", "content_type"]
                ),
            )
        )

LightFM_32_0.05
LightFM_32_0.01
LightFM_64_0.05
LightFM_64_0.01
CPU times: user 14min 19s, sys: 4.4 s, total: 14min 23s
Wall time: 1min 8s


In [62]:
models

{'ALS_32_True': <service.rec_models.als_model.ALSModel at 0x7f6eee35f220>,
 'ALS_64_True': <service.rec_models.als_model.ALSModel at 0x7f6ee8880970>,
 'ALS_32_False': <service.rec_models.als_model.ALSModel at 0x7f6ee889bbb0>,
 'ALS_64_False': <service.rec_models.als_model.ALSModel at 0x7f6ee88a32b0>,
 'LightFM_32_0.05': <service.rec_models.factoring_machine_model.FactoringMachineModel at 0x7f6ee88accd0>,
 'LightFM_32_0.01': <service.rec_models.factoring_machine_model.FactoringMachineModel at 0x7f6ee88ace20>,
 'LightFM_64_0.05': <service.rec_models.factoring_machine_model.FactoringMachineModel at 0x7f6ee88aca90>,
 'LightFM_64_0.01': <service.rec_models.factoring_machine_model.FactoringMachineModel at 0x7f6ee88ac4c0>}

In [63]:
TEST_USERS = test[Columns.User].unique()

In [64]:
np.array(TEST_USERS)

array([ 203219,  200197,   73446, ..., 1097559, 1097560, 1097561])

In [65]:
%%time
results = []
for model_name, model in models.items():
    model_quality = {'model': model_name}

    recos = model.model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    metric_values = calc_metrics(metrics, recos, test, train)
    model_quality.update(metric_values)
    results.append(model_quality)

CPU times: user 7min 17s, sys: 1.66 s, total: 7min 19s
Wall time: 7min 19s


In [66]:
df_quality = pd.DataFrame(results).T

df_quality.columns = df_quality.iloc[0]

df_quality.drop('model', inplace=True)

In [67]:
df_quality.style.highlight_max(color='lightgreen', axis=1)

model,ALS_32_True,ALS_64_True,ALS_32_False,ALS_64_False,LightFM_32_0.05,LightFM_32_0.01,LightFM_64_0.05,LightFM_64_0.01
Precision@1,0.084341,0.083620,0.062711,0.065150,0.082790,0.080326,0.080151,0.083196
Recall@1,0.044148,0.043696,0.031888,0.032925,0.042350,0.041880,0.041228,0.043408
Precision@2,0.070730,0.070066,0.056301,0.058408,0.072564,0.071813,0.071460,0.072190
Recall@2,0.071733,0.071094,0.056408,0.058101,0.073349,0.073425,0.072125,0.073584
Precision@3,0.061178,0.060899,0.052259,0.053885,0.064788,0.065039,0.063930,0.064796
Recall@3,0.091233,0.090791,0.077344,0.079156,0.097340,0.098376,0.095631,0.097746
Precision@4,0.054094,0.053760,0.048333,0.049802,0.057944,0.058566,0.057622,0.058556
Recall@4,0.106615,0.105840,0.093772,0.096146,0.114613,0.116882,0.113532,0.116801
Precision@5,0.048066,0.047741,0.044772,0.045778,0.052102,0.052741,0.051949,0.052586
Recall@5,0.117188,0.116252,0.107550,0.109260,0.127678,0.130278,0.126471,0.129738


In [70]:
model = models["LightFM_64_0.01"]
fake_user_id_series = new_user_id - 1
fake_user_id_sport = new_user_id - 2
fake_user_id_thriller = new_user_id - 3

---

Как видно, данному пользователю рекомендуют в основном триллеры, ужасы, детективы
Это не удивительно, ведь часто люди, которые смотрят триллеры, находят данные жанры схожими
Поэтому нашему любителю триллеров и рекомендуется такой набор фильмов
Более того, мы не видим тут в списке одни лишь триллеры, что хорошо, ведь это показатель отсутствия переобучения

В целом, рекомендация прошла успешна

In [71]:
recos = model.recommend(fake_user_id_thriller, 10)

acquired_items = items[items["item_id"].isin(recos)]
acquired_items


,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,genre
1006,3626,film,Развратные сестрички,Wet & Horny Stepsisters Deepthroat,2016.0,для взрослых,Великобритания,NaN,21.0,NaN,Джек Манила,NaN,Однажды летом к Майку приезжает на каникулы ег...,"2016, соединенное королевство, развратные, сес...",[для взрослых]
3975,8511,series,Охотники за бриллиантами,Okhotniki za brilliantami,2011.0,детективы,Россия,NaN,16.0,NaN,Александр Котт,"Александр Новин, Мария Аронова, Владимир Симон...",В ноябре 1980-го происходит ограбление вдовы А...,сериал мини-формата,[детективы]
5654,4484,film,Готика,Gothika,2003.0,"ужасы, триллеры, детективы","США, Франция, Канада, Испания",NaN,16.0,NaN,Матьё Кассовиц,"Холли Берри, Роберт Дауни мл., Чарльз С. Датто...",Женщина-психиатр оказывается пациенткой больни...,"амнезия, психиатрическая больница, пациент, до...","[ужасы, триллеры, детективы]"
6041,14100,film,Новенький,NaN,2017.0,"семейное, комедии",Россия,0.0,6.0,NaN,Мария Сопова,"Александр Алёшкин, Елена Лямина, Татьяна Плетн...","По статистике, семьи, имеющие больше одного ре...","Новенький, 2017, Россия","[семейное, комедии]"
6432,10104,film,Лучшее… от первого лица,NaN,2017.0,драмы,Россия,0.0,16.0,NaN,Александр Цой,"Кирилл Ковбас, Полина Ревис, Лада Марис, Галин...",Молодой человек знакомится с новой героиней ег...,"Лучшее…, от, первого, лица, 2017, Россия",[драмы]
6781,3634,film,Страшная воля богов,Kamisama no iu tôri,2014.0,"фантастика, зарубежные, триллеры, ужасы",Япония,NaN,18.0,NaN,Такаси Миике,"Ацуко Маэда, Дзинги Ириэ, Лили Фрэнки, Мио Юки...",Стоило японскому старшекласснику пожаловаться ...,"Страшная, воля, богов, 2014, Япония, куклы, св...","[фантастика, зарубежные, триллеры, ужасы]"
7216,630,film,Охотники за астероидами,Asteroid Chasers,2020.0,"зарубежные, для детей, документальное",Австралия,NaN,12.0,NaN,NaN,Tamara Davis,Научно-популярный фильм о малых телах Солнечно...,"Охотники, за, астероидами, 2020, Австралия, ко...","[зарубежные, для детей, документальное]"
11810,14814,film,Заражённая,Maggie,2014.0,"драмы, ужасы, триллеры","США, Швейцария",NaN,12.0,NaN,Генри Хобсон,"Эбигейл Бреслин, Джоэли Ричардсон, Арнольд Шва...","Девочка-подросток Мэгги заражается болезнью, м...","постапокалипсис, дистопия, зомби, 2015 год, Ви...","[драмы, ужасы, триллеры]"
14260,3435,series,Мой удивительный парень,NaN,2016.0,"мелодрамы, зарубежные, фэнтези, комедии",Китай,NaN,16.0,NaN,Шуй Цянь-Мо,"Ким Тхэ-хван, Лили Тиен, Пират Нитипаисалкул, ...",Молодая актриса вынуждена жить в одной квартир...,"Мой, удивительный, парень, 2016, Китай, друзья...","[мелодрамы, зарубежные, фэнтези, комедии]"
14417,12884,film,Любовь прет-а-порте,NaN,2017.0,"русские, комедии, мелодрамы",Россия,NaN,12.0,NaN,Макс Нардари,"Алессандро Борги, Андреа Прети, Антонио Де Мат...",Итальянский донжуан Джорджио из-за своего бесп...,"Любовь, прет-а-порте, 2017, Россия, любовь, мо...","[русские, комедии, мелодрамы]"


---

У данного пользователя в списке рекомендаций есть один фильм про спорт, и это уже хорошо.
Это можно объяснить тем, что фильмов с тематикой спорт не так уж и много, и люди, интересующиеся этим жанром часто вынуждены смотреть другие жанры.
Именно поэтому тут мы видим большое разнообразие.

In [72]:
recos = model.recommend(fake_user_id_sport, 10)

acquired_items = items[items["item_id"].isin(recos)]
acquired_items

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,genre
202,4880,series,Афера,Afera,2021.0,комедии,Россия,NaN,18.0,NaN,Михаил Старчак,"Сергей Степин, Игорь Царегородцев, Татьяна Лял...","Смотри:- как кино- как сериалКарантин окончен,...","Афера, Аферисты, Карантин, Пандемия, Карантин ...",[комедии]
767,15297,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",Россия,NaN,18.0,NaN,Александр Кириенко,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Успешный сексолог Алена уверена, что нашла фор...","Клиника счастья, Клиника, Счастье, Клиника сча...","[драмы, мелодрамы]"
1882,10440,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",Россия,NaN,18.0,NaN,Душан Глигоров,"Антон Васильев, Николай Шрайбер, Екатерина Оль...",Сергей Смирнов — один из лучших «охотников на ...,"хруст, хрусталь, хруста, хрус, полицейский, пе...","[триллеры, детективы]"
3875,6809,film,Дуров,NaN,2021.0,документальное,Россия,NaN,16.0,NaN,Родион Чепель,NaN,"Уникальная история о лидере нового поколения, ...","Компьютер, Монитор, Гений, Интервью, Предприни...",[документальное]
6346,9728,film,Гнев человеческий,Wrath of Man,2021.0,"боевики, триллеры","Великобритания, США",NaN,18.0,NaN,Гай Ричи,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...",Грузовики лос-анджелесской инкассаторской комп...,"ограбление, криминальный авторитет, месть, пер...","[боевики, триллеры]"
6501,13865,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",Россия,NaN,12.0,NaN,Тимур Бекмамбетов,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...",Военно-исторический блокбастер от режиссёров Т...,"Девятаев, Девятаева, Девят, Девя, Девята, Девя...","[драмы, военные, приключения]"
6689,4151,series,Секреты семейной жизни,NaN,2021.0,комедии,Россия,NaN,18.0,NaN,Шота Гамисония,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...",У Никиты и Полины всё начиналось прекрасно: об...,"брызги крови, кровь, жестокое обращение с живо...",[комедии]
7486,8636,film,Белый снег,Belyy sneg,2021.0,"драмы, спорт",Россия,NaN,6.0,NaN,Николай Хомерики,"Ольга Лерман, Федор Добронравов, Надежда Марки...",Отечественная спортивная драма про лыжницу и о...,"биография, занятие спортом, байопик, чемпионат...","[драмы, спорт]"
12050,3734,film,Прабабушка легкого поведения,Prababushka lyogkogo povedeniya,2021.0,комедии,Россия,NaN,16.0,NaN,Марюс Вайсберг,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...","1980 год, вся страна следит за событиями моско...",", 2021, россия, прабабушка, легкого, поведения",[комедии]
15234,142,film,Маша,Masha,2020.0,"драмы, триллеры",Россия,NaN,16.0,NaN,Анастасия Пальчикова,"Максим Суханов, Аня Чиповская, Полина Гухман, ...",13-летняя Маша растёт между боксёрским рингом ...,"Фильм Маша, Маша фильм 2021, Смотреть фильм Ма...","[драмы, триллеры]"


---

Ну и, наконец, последний аватар, который любит сериалы.
Сразу бросается в глаза, что сериалов ему по рекомендовалось больше, чем фильмов.
Опять же это хорошо, что есть разнообразие в виде фильмов. Это говорит об отсутствии переобучения.

In [73]:
recos = model.recommend(fake_user_id_series, 10)

acquired_items = items[items["item_id"].isin(recos)]
acquired_items


,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords,genre
202,4880,series,Афера,Afera,2021.0,комедии,Россия,NaN,18.0,NaN,Михаил Старчак,"Сергей Степин, Игорь Царегородцев, Татьяна Лял...","Смотри:- как кино- как сериалКарантин окончен,...","Афера, Аферисты, Карантин, Пандемия, Карантин ...",[комедии]
767,15297,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",Россия,NaN,18.0,NaN,Александр Кириенко,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Успешный сексолог Алена уверена, что нашла фор...","Клиника счастья, Клиника, Счастье, Клиника сча...","[драмы, мелодрамы]"
1882,10440,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",Россия,NaN,18.0,NaN,Душан Глигоров,"Антон Васильев, Николай Шрайбер, Екатерина Оль...",Сергей Смирнов — один из лучших «охотников на ...,"хруст, хрусталь, хруста, хрус, полицейский, пе...","[триллеры, детективы]"
3875,6809,film,Дуров,NaN,2021.0,документальное,Россия,NaN,16.0,NaN,Родион Чепель,NaN,"Уникальная история о лидере нового поколения, ...","Компьютер, Монитор, Гений, Интервью, Предприни...",[документальное]
6346,9728,film,Гнев человеческий,Wrath of Man,2021.0,"боевики, триллеры","Великобритания, США",NaN,18.0,NaN,Гай Ричи,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...",Грузовики лос-анджелесской инкассаторской комп...,"ограбление, криминальный авторитет, месть, пер...","[боевики, триллеры]"
6501,13865,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",Россия,NaN,12.0,NaN,Тимур Бекмамбетов,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...",Военно-исторический блокбастер от режиссёров Т...,"Девятаев, Девятаева, Девят, Девя, Девята, Девя...","[драмы, военные, приключения]"
6689,4151,series,Секреты семейной жизни,NaN,2021.0,комедии,Россия,NaN,18.0,NaN,Шота Гамисония,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...",У Никиты и Полины всё начиналось прекрасно: об...,"брызги крови, кровь, жестокое обращение с живо...",[комедии]
7279,9996,series,Немцы,Nemtsy,2021.0,драмы,Россия,NaN,16.0,NaN,Стас Иванов,"Евгений Коряковский, Анна Завтур, Антон Василь...",На журналиста-оппозиционера Антона Эбергарда с...,"немцы, немец, про немцев, по мотивам романа, п...",[драмы]
12050,3734,film,Прабабушка легкого поведения,Prababushka lyogkogo povedeniya,2021.0,комедии,Россия,NaN,16.0,NaN,Марюс Вайсберг,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...","1980 год, вся страна следит за событиями моско...",", 2021, россия, прабабушка, легкого, поведения",[комедии]
13615,2657,series,Подслушано,Podslushano,2021.0,"драмы, триллеры",Россия,NaN,16.0,NaN,Илья Куликов,"Александр Hовиков, Валентина Ляпина, Никита Па...",Смотри:- как кино- как сериалПодростковый псих...,"подслушано, подслушано в контакте, социальная ...","[драмы, триллеры]"
